In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mae
from tensorflow.keras.metrics import Mean
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Load and preprocess the dataset
df = pd.read_csv("D:\\UCD\\Research Paper\\Dataset M-En - Autencoder\\features_data1.csv")
df['Label'] = df['Label'].replace({0: 'Tr.B', 1: 'IoT.M', 2: 'IoT.M', 3: 'IoT.M', 4: 'IoT.B', 5: 'Tr.M'})

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)



In [49]:
# Select subset of data
df1 = df[:50000]

In [52]:
from collections import Counter
Counter(df1['Label'])

Counter({'IoT.M': 21312, 'Tr.B': 15735, 'Tr.M': 9928, 'IoT.B': 3025})

In [53]:
# Replace values in the 'Label' column
df1['Label'] = df1['Label'].replace({
    'Tr.B': 'Normal',
    'Tr.M': 'Normal',
    'IoT.B': 'Attack',
    'IoT.M': 'Attack'
})

In [54]:
df1.columns

Index(['Protocol', 'TCP', 'UDP', 'TTL', 'ACK', 'SYN', 'FIN', 'PSH', 'URG',
       'RST', 'SequenceNumber', 'AcknowledgmentNumber', 'PacketSize',
       'PayloadSize', 'Label', 'PacketSizeStd', 'PacketCount',
       'MostFreqSrcPort', 'MostFreqDstPort', 'AvgPacketSize',
       'MostFreqPacketizeFreq', 'AbnormalSizeFreq', 'SeqNumVariance',
       'ShortLivedConnections', 'RepeatedAttempts', 'ScanningActivity',
       'FlowRate', 'SourceEntropy', 'ConnectionErrors', 'DstPortEntropy',
       'AvgPacketNumber', 'SYNFrequency', 'ACKFrequency', 'TCPFrequency',
       'UDPFrequency', 'MostFreqProtocol', 'PacketSizeVar',
       'MostFreqPayloadSize', 'AvgPayloadSize'],
      dtype='object')

In [68]:
from collections import Counter
Counter(df1['Label'])

Counter({'Normal': 25663, 'Attack': 24337})

In [56]:
from collections import Counter
Counter(df1['Protocol'])

Counter({6: 30997, 17: 18999, 1: 4})

In [57]:
# Replace values in the 'Label' column
df1['Protocol'] = df1['Protocol'].replace({
    6 : 'TCP',
    17: 'UDP',
    1 : 'UDP',
    89: 'UDP'
})

In [58]:
from collections import Counter
Counter(df1['Protocol'])

Counter({'TCP': 30997, 'UDP': 19003})

In [59]:
from collections import Counter
Counter(df1['MostFreqProtocol'])

Counter({6: 30955, 17: 19042, 1: 3})

In [43]:
# Replace values in the 'Label' column
df1['MostFreqProtocol'] = df1['MostFreqProtocol'].replace({
    6 : 'TCP',
    17: 'UDP',
    1 : 'UDP',
})

In [60]:
from collections import Counter
Counter(df1['MostFreqProtocol'])

Counter({6: 30955, 17: 19042, 1: 3})

In [69]:
data=df1[:10000]

In [70]:
data

Protocol  TCP  UDP  TTL  ACK  SYN  FIN  PSH  URG  RST  ...  \
0         UDP    0    1   56    0    0    0    0    0    0  ...   
1         UDP    0    1   64    0    0    0    0    0    0  ...   
2         TCP    1    0   63    1    0    1    0    0    0  ...   
3         UDP    0    1   64    0    0    0    0    0    0  ...   
4         UDP    0    1   64    0    0    0    0    0    0  ...   
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
9995      TCP    1    0   64    1    0    0    0    0    0  ...   
9996      TCP    1    0   64    1    0    0    0    0    0  ...   
9997      UDP    0    1   64    0    0    0    0    0    0  ...   
9998      TCP    1    0   63    1    0    0    1    0    0  ...   
9999      TCP    1    0   64    1    0    0    0    0    0  ...   

      DstPortEntropy  AvgPacketNumber  SYNFrequency  ACKFrequency  \
0          10.068740          65864.0           4.0          18.0   
1           0.347783          56522.0           4.0        4566.0   
2           1.807315            174.0          26.0         161.0   
3           0.000565         314719.0           0.0           1.0   
4           0.014191         118214.0           0.0         177.0   
...              ...              ...           ...           ...   
9995        1.168364           1671.0           4.0        1669.0   
9996        0.774909          12124.0           2.0       12123.0   
9997        0.014191         118214.0           0.0         177.0   
9998        0.991983            696.0          24.0         683.0   
9999        0.137806         179967.0           0.0      179967.0   

     TCPFrequency  UDPFrequency  MostFreqProtocol  PacketSizeVar  \
0        0.000304      0.999605                17      77.112806   
1        0.080818      0.918651                17   54943.557520   
2        1.000000      0.000000                 6   69639.271676   
3        0.000003      0.999949                17       0.077939   
4        0.001497      0.998342                17     703.384883   
...           ...           ...               ...            ...   
9995     1.000000      0.000000                 6  509420.934556   
9996     1.000000      0.000000                 6  476850.261894   
9997     0.001497      0.998342                17     703.384883   
9998     0.998563      0.001437                 6  191909.340658   
9999     1.000000      0.000000                 6   15543.086474   

      MostFreqPayloadSize  AvgPayloadSize  
0                     516      515.888801  
1                     512      549.628552  
2                       0       95.885057  
3                     512      511.975162  
4                     512      512.009373  
...                   ...             ...  
9995                 1448      787.865350  
9996                 1448      941.373474  
9997                  512      512.009373  
9998                  888      498.653736  
9999                  512      520.488078  

[10000 rows x 39 columns]

In [47]:
# ML & DL Model Deployment on Network Traffic Dataset with K-Fold and Resource Monitoring and Full Output

import pandas as pd
import numpy as np
import time
import psutil
import GPUtil
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Machine Learning models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb

# Deep Learning models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


# Encode categorical features
le = LabelEncoder()
data['Protocol'] = le.fit_transform(data['Protocol'])
data['MostFreqProtocol'] = le.fit_transform(data['MostFreqProtocol'])

# Prepare features and target
X = data.drop(columns=['Label'])
y = data['Label']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Machine Learning Models
models_ml = {
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(probability=True),
    "Gradient Boosting": GradientBoostingClassifier(),
    "LightGBM": lgb.LGBMClassifier()
}

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("\n=== MACHINE LEARNING MODELS (K-Fold Evaluation) ===")
for name, model in models_ml.items():
    acc_scores = []
    all_reports = []
    start_time = time.time()
    for train_idx, test_idx in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc_scores.append(accuracy_score(y_test, preds))
        all_reports.append(classification_report(y_test, preds, output_dict=False))

    end_time = time.time()
    avg_acc = np.mean(acc_scores)
    cpu_percent = psutil.cpu_percent(interval=1)
    ram_percent = psutil.virtual_memory().percent
    #gpus = GPUtil.getGPUs()
    #gpu_info = f"GPU Memory Used: {gpus[0].memoryUsed} MB / {gpus[0].memoryTotal} MB" if gpus else "No GPU detected"

    print(f"\n{name} Average Accuracy: {avg_acc:.4f}")
    print(f"CPU Usage: {cpu_percent}% | RAM Usage: {ram_percent}% | Time: {end_time - start_time:.2f} sec")
    #print(gpu_info)
    print("\nClassification Reports from each fold:")
    for report in all_reports:
        print(report)




=== MACHINE LEARNING MODELS (K-Fold Evaluation) ===

Random Forest Average Accuracy: 0.9999
CPU Usage: 10.9% | RAM Usage: 48.1% | Time: 8.98 sec

Classification Reports from each fold:
              precision    recall  f1-score   support

      Attack       1.00      1.00      1.00      4885
      Normal       1.00      1.00      1.00      5115

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

              precision    recall  f1-score   support

      Attack       1.00      1.00      1.00      4885
      Normal       1.00      1.00      1.00      5115

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

              precision    recall  f1-score   support

      Attack       1.00      1.00      1.00      4865
      Normal       1.00      1.00      1.00      51

In [ ]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import time
import psutil
import os

data=df1[:200]
# Deep Learning models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Encode categorical features
le = LabelEncoder()
data['Protocol'] = le.fit_transform(data['Protocol'])
data['MostFreqProtocol'] = le.fit_transform(data['MostFreqProtocol'])

# Prepare features and target
X = data.drop(columns=['Label'])
y = data['Label']
y = le.fit_transform(y)

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Define model architectures
def create_dense_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_cnn_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Conv1D(32, 3, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_bilstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_attention_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    attention_probs = tf.keras.layers.Dense(input_dim, activation='softmax')(x)
    attention_mul = tf.keras.layers.Multiply()([inputs, attention_probs])
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(attention_mul)
    return tf.keras.Model(inputs, outputs)

# Model mapping
model_creators = {
    "Dense NN": create_dense_model,
    "1D CNN": create_cnn_model,
    "LSTM": create_lstm_model,
    "BiLSTM": create_bilstm_model,
    "Attention NN": create_attention_model
}

# K-Fold Deep Learning Evaluation
def evaluate_models(X, y, epochs=5, batch_size=64):
    input_dim = X.shape[1]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Ensure NumPy arrays for indexing
    X_np = X.to_numpy() if hasattr(X, "to_numpy") else np.array(X)
    y_np = y.to_numpy() if hasattr(y, "to_numpy") else np.array(y)

    for name, creator in model_creators.items():
        print(f"\n--- Evaluating {name} ---")
        acc_scores, times, cpu_usages = [], [], []

        for train_index, test_index in skf.split(X_np, y_np):
            X_train, X_test = X_np[train_index], X_np[test_index]
            y_train, y_test = y_np[train_index], y_np[test_index]

            model = creator(input_dim)
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            start_cpu = psutil.cpu_percent(interval=None)
            start_time = time.time()

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            elapsed_time = time.time() - start_time
            end_cpu = psutil.cpu_percent(interval=None)

            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            acc = accuracy_score(y_test, y_pred)

            acc_scores.append(acc)
            times.append(elapsed_time)
            cpu_usages.append(end_cpu - start_cpu)

        print(f"Avg Accuracy: {np.mean(acc_scores):.4f}")
        print(f"Avg CPU Usage: {np.mean(cpu_usages):.2f}%")
        print(f"Avg Time: {np.mean(times):.2f} sec")

# Example usage (uncomment and provide your real dataset)
evaluate_models(X, y)



--- Evaluating Dense NN ---
63/63 [==============================] - 0s 597us/step
Avg Accuracy: 0.5577
Avg CPU Usage: -1.04%
Avg Time: 1.02 sec

--- Evaluating 1D CNN ---
63/63 [==============================] - 0s 1ms/step
Avg Accuracy: 0.5094
Avg CPU Usage: 0.74%
Avg Time: 2.08 sec

--- Evaluating LSTM ---
63/63 [==============================] - 1s 4ms/step
Avg Accuracy: 0.9877
Avg CPU Usage: 7.64%
Avg Time: 8.81 sec

--- Evaluating BiLSTM ---
63/63 [==============================] - 1s 6ms/step
Avg Accuracy: 0.9958
Avg CPU Usage: 28.10%
Avg Time: 14.74 sec

--- Evaluating Attention NN ---
63/63 [==============================] - 0s 751us/step


In [ ]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import time
import psutil
import os
data=df1[:1000]
# Deep Learning models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Encode categorical features
le = LabelEncoder()
data['Protocol'] = le.fit_transform(data['Protocol'])
data['MostFreqProtocol'] = le.fit_transform(data['MostFreqProtocol'])

# Prepare features and target
X = data.drop(columns=['Label'])
y = data['Label']
y = le.fit_transform(y)

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Define model architectures
def create_dense_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_cnn_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Conv1D(32, 3, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_bilstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_attention_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    attention_probs = tf.keras.layers.Dense(input_dim, activation='softmax')(x)
    attention_mul = tf.keras.layers.Multiply()([inputs, attention_probs])
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(attention_mul)
    return tf.keras.Model(inputs, outputs)

# Model mapping
model_creators = {
    "Dense NN": create_dense_model,
    "1D CNN": create_cnn_model,
    "LSTM": create_lstm_model,
    "BiLSTM": create_bilstm_model,
    "Attention NN": create_attention_model
}

# K-Fold Deep Learning Evaluation
def evaluate_models(X, y, epochs=5, batch_size=64):
    input_dim = X.shape[1]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Ensure NumPy arrays for indexing
    X_np = X.to_numpy() if hasattr(X, "to_numpy") else np.array(X)
    y_np = y.to_numpy() if hasattr(y, "to_numpy") else np.array(y)

    for name, creator in model_creators.items():
        print(f"\n--- Evaluating {name} ---")
        acc_scores, times, cpu_usages = [], [], []

        for train_index, test_index in skf.split(X_np, y_np):
            X_train, X_test = X_np[train_index], X_np[test_index]
            y_train, y_test = y_np[train_index], y_np[test_index]

            model = creator(input_dim)
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            start_cpu = psutil.cpu_percent(interval=None)
            start_time = time.time()

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            elapsed_time = time.time() - start_time
            end_cpu = psutil.cpu_percent(interval=None)

            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            acc = accuracy_score(y_test, y_pred)

            acc_scores.append(acc)
            times.append(elapsed_time)
            cpu_usages.append(end_cpu - start_cpu)

        print(f"Avg Accuracy: {np.mean(acc_scores):.4f}")
        print(f"Avg CPU Usage: {np.mean(cpu_usages):.2f}%")
        print(f"Avg Time: {np.mean(times):.2f} sec")

# Example usage (uncomment and provide your real dataset)
evaluate_models(X, y)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import time
import psutil
import os
data=df1[:10000]
# Deep Learning models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Encode categorical features
le = LabelEncoder()
data['Protocol'] = le.fit_transform(data['Protocol'])
data['MostFreqProtocol'] = le.fit_transform(data['MostFreqProtocol'])

# Prepare features and target
X = data.drop(columns=['Label'])
y = data['Label']
y = le.fit_transform(y)

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Define model architectures
def create_dense_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_cnn_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Conv1D(32, 3, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_bilstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_attention_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    attention_probs = tf.keras.layers.Dense(input_dim, activation='softmax')(x)
    attention_mul = tf.keras.layers.Multiply()([inputs, attention_probs])
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(attention_mul)
    return tf.keras.Model(inputs, outputs)

# Model mapping
model_creators = {
    "Dense NN": create_dense_model,
    "1D CNN": create_cnn_model,
    "LSTM": create_lstm_model,
    "BiLSTM": create_bilstm_model,
    "Attention NN": create_attention_model
}

# K-Fold Deep Learning Evaluation
def evaluate_models(X, y, epochs=5, batch_size=64):
    input_dim = X.shape[1]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Ensure NumPy arrays for indexing
    X_np = X.to_numpy() if hasattr(X, "to_numpy") else np.array(X)
    y_np = y.to_numpy() if hasattr(y, "to_numpy") else np.array(y)

    for name, creator in model_creators.items():
        print(f"\n--- Evaluating {name} ---")
        acc_scores, times, cpu_usages = [], [], []

        for train_index, test_index in skf.split(X_np, y_np):
            X_train, X_test = X_np[train_index], X_np[test_index]
            y_train, y_test = y_np[train_index], y_np[test_index]

            model = creator(input_dim)
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            start_cpu = psutil.cpu_percent(interval=None)
            start_time = time.time()

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            elapsed_time = time.time() - start_time
            end_cpu = psutil.cpu_percent(interval=None)

            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            acc = accuracy_score(y_test, y_pred)

            acc_scores.append(acc)
            times.append(elapsed_time)
            cpu_usages.append(end_cpu - start_cpu)

        print(f"Avg Accuracy: {np.mean(acc_scores):.4f}")
        print(f"Avg CPU Usage: {np.mean(cpu_usages):.2f}%")
        print(f"Avg Time: {np.mean(times):.2f} sec")

# Example usage (uncomment and provide your real dataset)
evaluate_models(X, y)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import time
import psutil
import os
data=df1[:50000]
# Deep Learning models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Encode categorical features
le = LabelEncoder()
data['Protocol'] = le.fit_transform(data['Protocol'])
data['MostFreqProtocol'] = le.fit_transform(data['MostFreqProtocol'])

# Prepare features and target
X = data.drop(columns=['Label'])
y = data['Label']
y = le.fit_transform(y)

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Define model architectures
def create_dense_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_cnn_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Conv1D(32, 3, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_bilstm_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_attention_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    attention_probs = tf.keras.layers.Dense(input_dim, activation='softmax')(x)
    attention_mul = tf.keras.layers.Multiply()([inputs, attention_probs])
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(attention_mul)
    return tf.keras.Model(inputs, outputs)

# Model mapping
model_creators = {
    "Dense NN": create_dense_model,
    "1D CNN": create_cnn_model,
    "LSTM": create_lstm_model,
    "BiLSTM": create_bilstm_model,
    "Attention NN": create_attention_model
}

# K-Fold Deep Learning Evaluation
def evaluate_models(X, y, epochs=5, batch_size=64):
    input_dim = X.shape[1]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Ensure NumPy arrays for indexing
    X_np = X.to_numpy() if hasattr(X, "to_numpy") else np.array(X)
    y_np = y.to_numpy() if hasattr(y, "to_numpy") else np.array(y)

    for name, creator in model_creators.items():
        print(f"\n--- Evaluating {name} ---")
        acc_scores, times, cpu_usages = [], [], []

        for train_index, test_index in skf.split(X_np, y_np):
            X_train, X_test = X_np[train_index], X_np[test_index]
            y_train, y_test = y_np[train_index], y_np[test_index]

            model = creator(input_dim)
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            start_cpu = psutil.cpu_percent(interval=None)
            start_time = time.time()

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            elapsed_time = time.time() - start_time
            end_cpu = psutil.cpu_percent(interval=None)

            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            acc = accuracy_score(y_test, y_pred)

            acc_scores.append(acc)
            times.append(elapsed_time)
            cpu_usages.append(end_cpu - start_cpu)

        print(f"Avg Accuracy: {np.mean(acc_scores):.4f}")
        print(f"Avg CPU Usage: {np.mean(cpu_usages):.2f}%")
        print(f"Avg Time: {np.mean(times):.2f} sec")

# Example usage (uncomment and provide your real dataset)
evaluate_models(X, y)
